# Checking the aggregated data against its sources 

## Agreggating the source JSON files into a single dataframe and grouping based on the column filerName

In [53]:
import os
import pandas as pd


# Function to read all JSON files from a folder into a single DataFrame
def read_jsons_from_folder(folder_path):
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    df_list = []
    
    for file in all_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_json(file_path)
        df_list.append(df)
    
    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df
folder_path = 'data/campaign_finance'
combined_df = read_jsons_from_folder(folder_path)

In [59]:
#Display combined dataframe
#combined_df

In [55]:
# List of names to filter 'filerName' column by
names_to_filter = ['Maple', 'Steinberg', 'Guerra',
                   'Talamantes', 'Valenzuela', 'Kaplan', 
                   'Vang', 'Hume', 'Kennedy', 'Sterna', 
                   'Desmond', 'Jennings', 'Loloee', 'Frost']

# Create a function to check if any substring in a list appears in a string
def contains_any_substring(string, substrings):
    return any(sub.lower() in string.lower() for sub in substrings)

# Filter the DataFrame to include only rows where 'filerName' contains any of the specified names
filtered_df_partial_match = combined_df[combined_df['filerName'].apply(lambda x: contains_any_substring(x, names_to_filter))]

# Group by 'filerName' and aggregate the 'amount'
aggregated_by_filer_partial_match = filtered_df_partial_match.groupby('filerName').agg({'amount': 'sum'}).reset_index()

# Display the aggregated DataFrame
aggregated_by_filer_partial_match

,filerName,amount
0,Caity Maple for City Council 2022,189032.32
1,Caity Maple for City Council 2026,10271.00
2,Eric Guerra for City Council 2015,214918.62
3,Eric Guerra for City Council 2016,115796.62
4,Eric Guerra for City Council 2020,190794.00
5,Eric Guerra for City Council 2024,72191.03
6,"For A Better Sacramento, a Committee to Recall...",33312.85
7,Hume for Supervisor 2022,737219.93
8,Karina Talamantes for City Council 2022,310656.30
9,Karina Talamantes for City Council 2026,2000.00


## Comparing totals to the aggregated JSON data object flattened into a dataframe reveal discrepencies for every campaign total

In [58]:
aggregated_totals = pd.read_json('data/aggregated_totals.json')
# Flatten the 'data' column into its own DataFrame
data_flattened_df = pd.json_normalize(aggregated_totals['data'])

# Explode the 'committees' and 'contributors' columns to unpack the nested lists
exploded_committees = data_flattened_df.explode('committees')
exploded_contributors = data_flattened_df.explode('contributors')

# Flatten the dictionaries in the exploded 'committees' and 'contributors' columns
flattened_committees = pd.json_normalize(exploded_committees['committees'])
flattened_contributors = pd.json_normalize(exploded_contributors['contributors'])

# Merge the flattened DataFrames back with the original columns (excluding the exploded ones)
final_committees_df = pd.concat([exploded_committees.drop('committees', axis=1).reset_index(drop=True), flattened_committees], axis=1)
final_contributors_df = pd.concat([exploded_contributors.drop('contributors', axis=1).reset_index(drop=True), flattened_contributors], axis=1)

# Aggregate contributions by 'filerName'
aggregated_contributions = final_contributors_df.groupby([ 'filerName']).agg({'amount': 'sum'}).reset_index()

# Display the aggregated DataFrames
aggregated_contributions


,filerName,amount
0,Caity Maple for City Council 2022,179509
1,Eric Guerra for City Council 2015,275988
2,Eric Guerra for City Council 2016,144168
3,Eric Guerra for City Council 2020,134097
4,Eric Guerra for City Council 2024,34196
5,Hume for Supervisor 2022,733133
6,Karina Talamantes for City Council 2022,308886
7,Katie Valenzuela for Sacramento City Council 2020,71049
8,Katie Valenzuela for Sacramento City Council 2024,69389
9,Lisa Kaplan for City Council 2022,311348
